## Get SAR Asymmetries


In [2]:
# General
import glob
import re
import os.path
# import warnings
# warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
import copy

# Arrays & Displays
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
import pandas as pd

# Data treatment
from datetime import datetime
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

# Statistics
# from sklearn import linear_model, neighbors
# from pykalman import KalmanFilter

# Default parameters
mpl.rcParams.update({'font.size': 18})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [32]:
# Paths & Params
PTHS = {
    # 'dsg_pth': '/home/arthur/results/sar_extended/03_addIBT/dsg.nc',
    'dsg_pth': '/home/arthur/results/sar_extended/05_addPRIMED/dsg.nc',
    # 'sar_dir': '/home/arthur/data/cyclobs/RCM/sar_files/',
    'sar_dir': '/home/arthur/data/tropical_cyclones/SAR/S1_RS2_RCM/sar_files/',
    'sav_dir': '/home/arthur/results/TCsLifeMonitFromObs/winddirection/01_getSARasymmetries/',
}

In [28]:
def wn1(thetas, a, phi, c):
    return a * np.cos(thetas + phi) + c

def get_wn1(thetas, wind_speed, initial_condition=15, upper_bound=80):
    '''
    initial_condition: initial conidition for the amplitude and the constant c (just put Vplus)
    upper_bound: upper bound for the the amplitude and the constant c (just put Vmax)
    '''
    # Fitting process
    popt, pcov = curve_fit(
        f=wn1,
        xdata=thetas,
        ydata=wind_speed,
        p0=[initial_condition, np.pi / 2, initial_condition],
        bounds=([0.1, 0., 0.1], [upper_bound, np.pi, upper_bound]) # ([lower bounds], [upper bounds])
    )
    a, phi, c = popt[0], popt[1], popt[2]
    return a, phi, c

def I(Rs, Vs, fcor):
    dr   = np.diff(Rs[0, :])[0] # meters
    ksi  = 2 * Vs / Rs + fcor
    zeta = np.gradient(Vs, dr, axis=1) + Vs / Rs + fcor
    return np.sqrt(ksi * zeta)

def holland_profile(r, lat, B, Vmin, Rmax, Vmax):
    '''We assume that rho is constant and equals 1.15 kg.m-3'''
    fcor   = abs(coriolis(lat))
    rho    = 1.15
    r      = r.astype(float) + 0.001           # To avoid dividing by zero
    r_star = (Rmax / r) ** B
    V      = r * 0.
    V      = Vmin + np.sqrt( ((Vmax - Vmin) ** 2) * r_star * np.exp(1 - r_star) + (r * fcor / 2) ** 2) - (r * fcor / 2)
    return V

def coriolis(lat):
    '''Latitude must be in degrees.'''
    Omega = 7.2921e-5                             # Earth rotation vector
    fcor  = 2 * Omega * np.sin(lat * np.pi / 180) # Coriolis parameter assuming it's constant 
    return fcor

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Reference grids
rs           = np.linspace(100, 5e5, 5000)
ths          = np.linspace(0, 2 * np.pi, 361)                         # in radians
Rs, THs      = np.meshgrid(rs, ths)                                   # shape (361, 5000)

In [29]:
# Open data
dsg = xr.open_dataset(PTHS['dsg_pth'])

In [30]:
### FILTERS
filters = {
    'correct_center': (dsg.center_quality_flag < 2),
    'uncropped'     : (dsg.percent_outside < 5),
    'tropical_lat'  : (abs(dsg.lat) < 30),
    # 'northern_hem'  : (dsg.lat > 0), # remove southern hemisphere or not?
    'strong_storm'  : (dsg.vmx >= 33),
    'small_rmx'     : (dsg.rmx <= 100000),
    'far_from_coast': (dsg.dist2coast >= dsg.r0),   

    'low_vws':        (dsg.speed > 2 * dsg.vws_pmd), # new filter
}

kept_storms = np.logical_and.reduce([filters[e] for e in filters.keys()])
dsg['keep'] = xr.DataArray(
    data=kept_storms,
    coords={'time': dsg.time}
)

dsg_valid = dsg.where(dsg.keep, drop=True).drop('keep')
print('Nb of valid cases: {}'.format(len(dsg_valid.time)))

Nb of valid cases: 23


/tmp/ipykernel_37214/1245524597.py:20: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  dsg_valid = dsg.where(dsg.keep, drop=True).drop('keep')


In [31]:
# Initialize
listOfDatasets = []

# Iterate
for t in tqdm(range(len(dsg_valid.time))):
    # Open file
    ds  = dsg_valid.isel(time=t)
    dsp = xr.open_dataset(PTHS['sar_dir'] + ds.file_tcva.item()).isel(time=0)
    
    # Fixed parameters
    vmx = float(ds.vmx_hol)
    rmx = float(ds.rmx_hol)
    vmn = float(ds.vmn_hol)
    B   = float(ds.B_hol)
    lat = float(ds.lat)
    fcr = abs(float(ds.fcor))

    # Holland with fixed parameters
    V   = holland_profile(rs, lat, B, vmn, rmx, vmx)
    Vs  = np.stack([V for i in range(np.shape(THs)[0])]) # Shape (361, 5000)

    # I ~ V/r radii
    try:
        I_Vr         = (I(Rs, Vs, fcr) - Vs / rs)[0, :]    # I - V/r
        r_stt, r_end = rs[I_Vr <= 0][0], rs[I_Vr <= 0][-1] # Define the bounds
    except IndexError:
        r_stt = r_end = np.nan

    # Prepare averagings
    timescale   = 1 / I(Rs, Vs, fcr)[0, :]             # time needed for the BL to develop
    radialscale = timescale * Vs[0, :] / 5             # radial distance travelled by an air parcel in the meantime, considering that U/V = 5
    tangenscale = timescale * Vs[0, :]                 # tangential distance travelled by an air parcel in the meantime
    thetascale  = 360 * tangenscale / (2 * np.pi * rs) # convert this tangential distance to theta
    thetascale[np.isnan(thetascale)] = 9999            # # happens when time 1/I explodes, so put a high value to enter the ValueError exception below (AS2 and AS3 computation)

    # print(t, ds.file.item())

    # x-axis
    # xs = np.linspace(ds.rmx, 2 * ds.rps, 50)
    # xs = np.linspace(ds.rmx, 2 * ds.rps, 10)
    # xs = np.linspace(1000, 250000, 250)[::100]
    xs = np.linspace(1000, 250000, 250)
    
    # Asymmetries
    ASS  = []
    AS2  = []
    AS3  = []
    ASQ  = []
    rads = []
    ERR  = []
    VTS  = []
    for rad in tqdm(xs):
        try:
            # dsp.sel(rad=rad, method='nearest').wind_speed.fillna(dsp.sel(rad=rad, method='nearest').wind_speed.mean(skipna=True)).plot()
            # ass, _, _ = get_wn1(np.deg2rad(dsp.theta), dsp.sel(rad=rad, method='nearest').wind_speed.fillna(dsp.sel(rad=rad, method='nearest').wind_speed.mean(skipna=True)), float(ds.vps), float(ds.vmx))
            valid_dsp     = dsp.sel(rad=rad, method='nearest').dropna(subset=['wind_speed'], dim='theta')
            ass, phi, c   = get_wn1(np.deg2rad(valid_dsp.theta), valid_dsp.wind_speed, float(ds.vps), float(ds.vmx))
            error         = float(rmse(wn1(np.deg2rad(valid_dsp.theta), ass, phi, c), valid_dsp.wind_speed)) 
            ASS.append(ass)
            ERR.append(error)
            VTS.append(len(valid_dsp.theta))

            # Preparing averagings
            avg_on_u = radialscale[rs >= rad][0]
            avg_on_v = int(thetascale[rs >= rad][0])

            try:
                # Average on theta
                valid_dsp2      = dsp.sel(rad=rad, method='nearest').rolling(theta=avg_on_v, center=True, min_periods=2).mean().dropna(subset=['wind_speed'], dim='theta')
                as2, phi2, c2   = get_wn1(np.deg2rad(valid_dsp2.theta), valid_dsp2.wind_speed, float(ds.vps), float(ds.vmx))
                AS2.append(as2)

                # Average on theta AND r
                valid_dsp3      = dsp.sel(rad=slice(rad - avg_on_u, rad + avg_on_u))[['wind_speed']].mean(dim='rad', skipna=True).rolling(theta=avg_on_v, center=True, min_periods=2).mean().dropna(dim='theta')
                as3, phi3, c3   = get_wn1(np.deg2rad(valid_dsp3.theta), valid_dsp3.wind_speed, float(ds.vps), float(ds.vmx))
                AS3.append(as3)
                
            except ValueError:
                # happens when time 1/I explodes, such that to average on theta we would average on more than 361 values of theta!
                as2 = as3 = np.nan
                AS2.append(as2)
                AS3.append(as3)

            # Add radius
            rads.append(rad)

        except RuntimeError:
            print('RuntimeError when computing the asymmetries.')
            break
        # Asymmetries with the quantiles
        a_qtl = dsp.sel(rad=rad, method='nearest').wind_speed.quantile(0.9) - dsp.sel(rad=rad, method='nearest').wind_speed.quantile(0.1)
        ASQ.append(a_qtl)
        
    # Inertial neutrality?
    dr                 = np.diff(Rs[0, :])[0] # meteRs
    dM_dr              = Vs + Rs * np.gradient(Vs, dr, axis=1) + fcr * Rs
    beta               = dM_dr[0, :] / rs
    u_in               = (0.5 * Vs[0, :] / (2 * float(ds.rps))) * rs
    u_out              = 0.5 * Vs[0, :]
    u                  = np.ones_like(Vs[0, :])
    msk                = rs <= 2 * float(ds.rps)
    u[msk]             = u_in[msk]   # Assume that u/v ~ r   on [0, 2R+]
    u[~msk]            = u_out[~msk] # Assume that u/v = 0.5 on [2R+, infty]
    idxs               = np.searchsorted(rs, xs)
    secnd_term_eq5_k01 = u[idxs] * beta[idxs]
    first_term_eq5_k01 = (Vs[0, idxs] / rs[idxs]) * (np.array(AS3) / (2 * np.pi))
    
    # Create dataset
    dsa = xr.Dataset(
        data_vars={
            'wn1':            ('rad', ASS),
            'wn1_avgTH':      ('rad', AS2),
            'wn1_avgTHandR':  ('rad', AS3),
            'wn1_err':        ('rad', ERR),
            'valid_thetas':   ('rad', VTS),
            'quantile_diff':  ('rad', ASQ),
            'inertial_ratio': ('rad', first_term_eq5_k01 / secnd_term_eq5_k01),
            'translat_ratio': ('rad', float(ds.speed) / Vs[0, idxs]),
            'I_Vr1':          r_stt,
            'I_Vr2':          r_end,
        },
        coords={
            'time': ds.time,
            'rad': xs
        }
    )
    listOfDatasets.append(dsa)

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [33]:
# Concatenate dataset
dsga = xr.concat(listOfDatasets, dim='time')
# Add it to dsg_valid
dsgm = xr.merge([dsg_valid, dsga]) # dsg merged

In [34]:
# Attributes
dsgm.rad.attrs = {'long_name': 'Radius, or distance from TC center', 'description': 'Radii used to compute the asymmetric WN1s', 'units': 'meters'}
dsgm.wn1.attrs = {
    'long_name':   'Wave Number 1', 
    'description': 'Amplitude of a cosine fitted, at each radius, on the SAR azimuthal wind speed', 
    'method':      'Both the phase and and the intercept are let as free parameters during fitting procedure',
    'units':       'm/s'          
}
dsgm.wn1_avgTH.attrs = {
    'long_name':   'Wave Number 1 when SAR azimuthal wind is averaged on the theta dimension', 
    'description': 'Amplitude of a cosine fitted, at each radius, on the SAR azimuthal wind speed avg. on the theta dimension, according to the development time of the BL', 
    'method':      'Both the phase and and the intercept are let as free parameters during fitting procedure',
    'units':       'm/s'          
}
dsgm.wn1_avgTHandR.attrs = {
    'long_name':   'Wave Number 1 when SAR azimuthal wind is averaged on both the theta and radius dimensions', 
    'description': 'Amplitude of a cosine fitted, at each radius, on the SAR azimuthal wind speed avg. on both the theta and radius dimensions, according to the development time of the BL', 
    'method':      'Both the phase and and the intercept are let as free parameters during fitting procedure',
    'units':       'm/s'          
}
dsgm.wn1_err.attrs        = {'long_name': 'WN1 RMSE', 'description': 'RMSE between the fitted WN1 (non-averaged version) and the SAR azimuthal wind speed', 'units': 'm/s'}
dsgm.valid_thetas.attrs   = {'long_name': 'Valid thetas', 'description': 'Nb of valid wind speed values on the theta dimension (between 0 and 360)', 'units': 'm/s'}
dsgm.quantile_diff.attrs  = {'long_name': 'Asymmetric quantile difference', 'description': 'Diff. between the 90%- and 10%-quantile of the SAR azim. wind speed', 'units': 'm/s'}
dsgm.inertial_ratio.attrs = {
    'long_name':   'Ratio to assess the inertial neutrality of the SAR wind profile', 
    'description': 'Ratio of the 1st term of Eq.5 from K01, to the 2nd term of the equation', 
    'method':      'u/v is assumed linear on [0, 2R+] and then constant = 1/2'        
}
dsgm.translat_ratio.attrs = {
    'long_name':   'Translation speed ratio', 
    'description': 'Ratio of the translation speed over the SAR wind speed', 
    'units': 'm/s'          
}
dsgm.I_Vr1.attrs = {'long_name':   'First radius where I ~ V/r (whenever it exists)', 'units': 'meters'}
dsgm.I_Vr2.attrs = {'long_name':   'Second radius where I ~ V/r (whenever it exists)', 'units': 'meters'}

In [37]:
# # Save
# dsgm[['name', 'id', 'file', 'file_tcva', 'basin', 'id_ibt']] = dsgm[['name', 'id', 'file', 'file_tcva', 'basin', 'id_ibt']].astype(str)
# dsgm.to_netcdf(PTHS['sav_dir'] + 'dsga.nc')